In [ ]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import re
import pickle

In [ ]:
raw_data=pd.read_csv("./IMDB Dataset.csv")
raw_data=raw_data.drop_duplicates()
raw_data.isna

In [ ]:
raw_data['sentiment']=pd.get_dummies(raw_data['sentiment'])['positive'].values
raw_data

In [ ]:
stemmer=PorterStemmer()
vectorizer=TfidfVectorizer(max_features=5000,analyzer='word', encoding='utf-8')

In [ ]:
stopwords=set(nltk.corpus.stopwords.words('english'))
c=1
cleaned_text=[]
c+=1
for i in raw_data['review'].values.tolist():
    i=BeautifulSoup(i,'lxml').text
    i=re.sub('[^a-zA-z]',' ',i).lower()
    words=[]
    for j in i.split():
        if j not in stopwords:
            words.append(stemmer.stem(j))
    cleaned_text.append(' '.join(words))

In [ ]:
vectors=vectorizer.fit_transform(cleaned_text)

with open('vectorizer.pickle','wb') as f:
    pickle.dump(vectorizer,f,protocol=2)

In [ ]:
model=RandomForestClassifier()
model.fit(vectors,raw_data['sentiment'])


In [ ]:
def clean_text(review):
    review=BeautifulSoup(review,'lxml').text
    review=re.sub('[^a-zA-z]',' ',review).lower()
    words=[]
    for j in review.split():
        if j not in stopwords:
            words.append(stemmer.stem(j))
    return ' '.join(words)

def predict_class(review):
    review=vectorizer.transform([clean_text(review)])
    result=model.predict(review)[0]
    return result

In [ ]:
predict_class('terriblly good')


In [ ]:
with open('model.pickle','wb') as f:
    pickle.dump(model,f,protocol=2)